# Mass conversion between different mass definitions
## Mass conversion between spherical overdensity mass definitions

In this notebook, we demonstrates how to convert the mass and concentration between various mass definitions (going from $200m$ to $500c$ in this example), and related functionalities, using both the object-oriented and functional interfaces of the code.

In [ ]:
import os

os.environ["CLMM_MODELING_BACKEND"] = (
    "ccl"  # here you may choose ccl, nc (NumCosmo) or ct (cluster_toolkit)
)
import clmm
import matplotlib.pyplot as plt
import numpy as np

We define a cosmology first:

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.27 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

In [ ]:
# first SOD definition
m1 = 1e14
c1 = 4
massdef1 = "mean"
delta_mdef1 = 200
halo_profile_model1 = "nfw"
# cluster redshift
z_cl = 0.4

In [ ]:
# get a value for alpha
ein_def1 = clmm.Modeling(
    massdef=massdef1, delta_mdef=delta_mdef1, halo_profile_model="einasto"
)
ein_def1.set_mass(m1)
ein_def1.set_concentration(c1)

alpha = ein_def1.get_einasto_alpha(z_cl)
alpha

In [ ]:
var_mod = {
    "massdef2": ("mean", "critical"),
    "delta_mdef2": (150, 200, 500),
    "halo_profile_model2": ("nfw", "einasto", "hernquist"),
}


def loop_keys(keys_use, current_info, cfg_dict, list_cases):
    if len(keys_use) == 0:
        list_cases.append(
            current_info
            | {
                "data": clmm.convert_profile_mass_concentration(
                    mdelta=m1,
                    cdelta=c1,
                    redshift=z_cl,
                    cosmo=cosmo,
                    massdef=massdef1,
                    delta_mdef=delta_mdef1,
                    halo_profile_model=halo_profile_model1,
                    alpha=alpha,
                    **current_info
                )
            }
        )
        return

    new_keys = list(keys_use)
    key = new_keys.pop(0)
    for value in cfg_dict[key]:
        new_info = current_info | {key: value}
        loop_keys(new_keys, new_info, cfg_dict, list_cases)


new_defs3 = []
loop_keys(list(var_mod.keys()), {}, var_mod, new_defs3)

In [ ]:
from paper_formating import prep_plot

In [ ]:
_config = {
    "massdef2": {"mean": {"s": 20}, "critical": {"s": 100}},
    "delta_mdef2": {150: {"c": "C0"}, 200: {"c": "C1"}, 500: {"c": "C2"}},
    "halo_profile_model2": {
        "nfw": {"marker": ".", "lw": 0.5, "edgecolors": "0", "zorder": 10},
        "einasto": {"marker": "^", "lw": 0.5, "edgecolors": "0", "zorder": 2},
        "hernquist": {"marker": "v", "lw": 0.5, "edgecolors": "0", "zorder": 2},
    },
}
_leg = {
    "massdef2": r"$\rho$: ",
    "delta_mdef2": r"$\Delta$: ",
    "halo_profile_model2": "",
}
_leg_val = {
    "nfw": "NFW",
    "einasto": "Einasto",
    "hernquist": "Hernquist",
}
fig = prep_plot(figsize=(9, 9))
ax = plt.axes()
for _case in new_defs3:
    _kwargs = {}
    label = []
    for key, value in _case.items():
        if key != "data":
            _kwargs.update(_config[key][value])
            label.append(_leg[key] + f"{_leg_val.get(value, value)}")
    label = ", ".join(label)
    ax.scatter(*_case["data"], **_kwargs, label=label)
ax.legend(ncols=3)
# ax.set_xscale("log")
# """
handles, labels = ax.get_legend_handles_labels()

leg1 = ax.legend(
    handles[:3],
    [lab.split(", ")[-1] for lab in labels[:3]],
    loc=(0.38, 0.8),
    fontsize=8,
)
leg2 = ax.legend(
    handles[:9:3],
    [lab.split(", ")[1] for lab in labels[:9:3]],
    loc=(0.015, 0.6),
    fontsize=8,
)
leg3 = ax.legend(
    handles[::9],
    [lab.split(", ")[0] for lab in labels[::9]],
    loc=(0.015, 0.83),
    fontsize=8,
)
ax.add_artist(leg1)
ax.add_artist(leg2)
# """
ax.set_xlabel("$M$ [$M_\odot$]")
ax.set_ylabel("concentration")
fit_kwargs = {"color": "r", "zorder": 1, "lw": 1}  # , "ls": "--"}
ax.axvline(1e14, **fit_kwargs)
ax.axhline(4, **fit_kwargs)
ax.grid(lw=0.5)
ax.minorticks_on()
ax.grid(which="minor", lw=0.1)
ax.set_axisbelow(True)
plt.tight_layout()
plt.savefig("mass_conversion.png")